In [32]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations
import pandas as pd
import numpy as numpy
from datetime import timedelta,datetime
from tqdm import tqdm_notebook
%matplotlib inline

In [33]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


In [34]:
data = pd.read_csv('bitcoin_cash_price.csv')
data.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,"Feb 20, 2018",1543.27,1569.03,1414.35,1418.73,"820,947,000","26,199,800,000"
1,"Feb 19, 2018",1483.34,1553.81,1483.34,1534.77,"578,906,000","25,179,700,000"
2,"Feb 18, 2018",1552.10,1641.40,1428.49,1487.46,"907,873,000","26,344,200,000"
3,"Feb 17, 2018",1548.48,1568.64,1517.14,1551.39,"641,719,000","26,280,100,000"
4,"Feb 16, 2018",1373.16,1558.66,1369.68,1552.20,"961,010,000","23,302,000,000"


In [35]:
data.columns= data.columns.str.lower()
data['date'] = data['date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d'))

In [36]:
model_data = data[data['date']>= '2017-01-01']

In [37]:
M = Model(data= model_data,variable='close',predicted_period_size=7)

In [38]:
M

In [39]:
M.build()

In [40]:
M.predict(denormalized=True)

array([1469.1827, 1478.0405, 1438.771 , 1474.5562, 1352.7963, 1454.8617,
       1437.2932], dtype=float32)

In [41]:
M.evaluate()

{'mse': 0.01, 'rmse': 163.06, 'mape': 7.9}

In [42]:
M.save('bitcoin_model_prod_v0.h5')

In [43]:
M = Model(path='bitcoin_model_prod_v0.h5',data=model_data,variable='close',predicted_period_size=7)

In [44]:
M.predict(denormalized=True)

array([1469.1827, 1478.0405, 1438.771 , 1474.5562, 1352.7963, 1454.8617,
       1437.2932], dtype=float32)

In [48]:
M1 = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [50]:
M1.build()

In [51]:
M1.train()

In [96]:
results = []
for i in range(1,11):
    M1.train(model_data[0*7:7*48 + 7])
    results.append(M1.evaluate())

In [97]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 137.16, 'mape': 6.41}
Week 42: {'mse': 0.0, 'rmse': 137.07, 'mape': 6.41}
Week 43: {'mse': 0.0, 'rmse': 137.67, 'mape': 6.42}
Week 44: {'mse': 0.0, 'rmse': 136.93, 'mape': 6.41}
Week 45: {'mse': 0.0, 'rmse': 137.18, 'mape': 6.41}
Week 46: {'mse': 0.0, 'rmse': 136.32, 'mape': 6.44}
Week 47: {'mse': 0.0, 'rmse': 137.73, 'mape': 6.42}
Week 48: {'mse': 0.0, 'rmse': 137.02, 'mape': 6.41}
Week 49: {'mse': 0.0, 'rmse': 137.33, 'mape': 6.41}
Week 50: {'mse': 0.0, 'rmse': 136.99, 'mape': 6.41}


In [85]:
M1.predict(denormalized=True)

array([1419.5465, 1097.5981, 1291.3864, 1437.2444, 1514.8063, 1438.3875,
       1365.1937], dtype=float32)

In [86]:
old_data = model_data[0*7:7*48 + 7]

In [87]:
new_data = model_data[0*7:7*49 + 7]

In [88]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [89]:
M2.build()
M1.train()

In [90]:
M2.predict(denormalized=True)

array([1418.257 , 1429.23  , 1419.965 , 1433.0006, 1479.0697, 1398.4965,
       1423.343 ], dtype=float32)

In [91]:
M3 = Model(data=new_data,
          variable='close',
          predicted_period_size=7)

In [92]:
M3.build()
M3.train()

In [93]:
M3.predict(denormalized=True)

array([1414.1393, 1107.0254, 1287.3068, 1438.5845, 1512.0979, 1435.444 ,
       1363.9971], dtype=float32)